In [1]:
import pandas as pd
import subprocess
import sys
import os
import pathlib
import time
import pandas as pd
import numpy as np
from dataclasses import dataclass
from datetime import datetime
import argparse

# Level 2: Processing CSV

## Globals

In [2]:
DIR = "data_processed/"

## Data validation

In [3]:
def load_csv(filename: str) -> pd.DataFrame:
    """
    Load a CSV file into a pandas DataFrame
    """

    df = pd.read_csv(filename)
    df['time'] = pd.to_datetime(df['time'])
    return df

def verify_df_column_names(df: pd.DataFrame) -> None:
    """
    Verify that the dataframe has the required columns
    """

    REQUIRED_COLS = ['time', 'event_type', 'channel', 'value']

    cols = df.columns
    for col in REQUIRED_COLS:
        if col not in cols:
            raise ValueError(f"Column {col} not found in dataframe")

def verify_df_sorted(df: pd.DataFrame) -> None:
    """
    Verify that the dataframe is sorted by "time"
    """
    # Find rows where the 'time' is decreasing from the previous row
    not_sorted_mask = df['time'].diff().dt.total_seconds() < 0

    # The first row can't be "not sorted" by definition, so we can exclude it from the mask
    not_sorted_mask.iloc[0] = False

    # Filter the DataFrame to find the not sorted rows
    not_sorted_rows = df[not_sorted_mask]

    if not df['time'].is_monotonic_increasing:
        raise ValueError(f"Dataframe is not sorted by time:\n{not_sorted_rows}")

def verify_df_time_group(df_group: pd.DataFrame) -> None:
    """
    Verify that the group has the correct number of events and channels
    """

    # verify counts: 9 for protons, 9 for electrons, 31 for directional 
    counts = df_group['event_type'].value_counts()
    if counts.get('d', 0) != 31:
        raise ValueError(f"Directional events count is not 31 for {df_group['time'].iloc[0]}")
    if counts.get('p', 0) != 9:
        raise ValueError(f"Proton events count is not 9 for {df_group['time'].iloc[0]}")
    if counts.get('e', 0) != 9:
        raise ValueError(f"Electron events count is not 9 for {df_group['time'].iloc[0]}")
    
    # verify counts: 49 total, no more no less
    if len(df_group) != 49:
        raise ValueError(f"Total events count is not 49 for {df_group['time'].iloc[0]}")
    
    # check for "channel" uniqueness for each "event_type"
    for event_type in ['d', 'e', 'p']:
        filtered_group = df_group[df_group['event_type'] == event_type]
        max_channel = {'d': 31, 'e': 9, 'p': 9}[event_type]
        if not all(filtered_group['channel'].value_counts() == 1) or sorted(filtered_group['channel'].unique()) != list(range(1, max_channel + 1)):
            raise ValueError(f"Channel uniqueness for {event_type} events is not correct for {df_group['time'].iloc[0]}")

def verify_df_time_diffs(df: pd.DataFrame, 
                         max_diff_tolerance: np.timedelta64 = np.timedelta64(90, 's'), 
                         min_diff_tolerance: np.timedelta64 = np.timedelta64(500, 'ms')) -> None:
    """
    Verify that the time differences between events are within tolerance.
    If time diff >= max_diff_tolerance, just prints the warning (data holes are permitted).
    If time diff <= min_diff_tolerance, raises an exception (possible floating point errors).
    
    Assumes that the dataframe is non-decreasingly sorted by "time".  
    
    There may me multiple groups of events with the same time.
    
    Args:
        df (pd.DataFrame): input dataframe with "time" column
        max_diff_tolerance (np.timedelta64, optional): max time difference tolerance in ms (warning only)
        min_diff_tolerance (np.timedelta64, optional): min time difference tolerance in ms (exception)

    Raises:
        ValueError: when time differences < min_diff_tolerance (possible floating point errors)
    """

    # get all unique "time" values in df
    times = df['time'].unique()

    # calc time diffs
    time_diffs = np.diff(times)

    # check if all time diffs are not larger than the tolerance
    checks = max_diff_tolerance > time_diffs
    if not all(checks):
        # find all indexes of unmet conditions
        indexes = np.where(checks == False)[0]

        # create a dataframe of times
        df_times = pd.DataFrame(times, columns=["time"])

        # find all holes
        holes = [f"{df_times.iloc[i]['time']} and {df_times.iloc[i + 1]['time']}" for i in indexes]
        
        print("Found time holes out of tolerance at times:", *holes, sep='\n\t')


    # check if all time diffs are not smaller than the tolerance
    # (possible floating point errors)
    checks = min_diff_tolerance < time_diffs
    if not all(checks):
        # find all indexes of unmet conditions
        indexes = np.where(checks == False)[0]

        # create a dataframe of times
        df_times = pd.DataFrame(times, columns=["time"])

        # find all too close values
        too_close = [f"{df_times.iloc[i]['time']} and {df_times.iloc[i + 1]['time']}" for i in indexes]
        
        raise ValueError(
            "Found time values too close to each other at times " +
            "(possible floating point errors):\n\t" +
            "\n\t".join(too_close))

def verify_df(df: pd.DataFrame) -> None:
    """
    Verify the integrity of the dataframe
    """

    if df.empty:
        raise ValueError("Dataframe is empty")

    verify_df_column_names(df)

    verify_df_sorted(df)

    # df.groupby('time').apply(verify_df_time_group)

    verify_df_time_diffs(df)

def load_and_verify_csv(filename: str) -> pd.DataFrame:
    """
    Load a CSV file into a pandas DataFrame and verify its integrity
    """

    df = load_csv(filename)

    verify_df(df)

    return df

def load_and_verify_csvs(filenames: list) -> list:
    """
    Load a list of CSV files into a list of pandas DataFrames and verify their integrity
    """

    dfs = []
    for filename in filenames:
        print("Verifying", filename, "...")
        df = load_and_verify_csv(filename)
        dfs.append(df)

    print("SUCCESS")
    return dfs

In [4]:
dfs = load_and_verify_csvs([DIR + str(i) + ".csv" for i in range(0, 363)])

Verifying data_processed/0.csv ...
Found time holes out of tolerance at times:
	2023-04-16 14:07:25.507109 and 2023-04-16 14:32:35.487163
	2023-04-16 15:02:52.503457 and 2023-04-16 15:58:35.491363
	2023-04-16 16:00:25.497549 and 2023-04-16 16:07:35.382710
	2023-04-16 16:37:52.384268 and 2023-04-16 16:54:35.485444
	2023-04-16 16:56:25.492226 and 2023-04-16 17:10:35.498404
Verifying data_processed/1.csv ...
Found time holes out of tolerance at times:
	2023-04-16 14:07:25.507000 and 2023-04-16 14:32:35.487000
	2023-04-16 15:02:52.504000 and 2023-04-16 15:58:35.492000
	2023-04-16 16:00:25.499000 and 2023-04-16 16:07:35.384000
	2023-04-16 16:37:52.385000 and 2023-04-16 16:54:35.487000
	2023-04-16 16:56:25.493000 and 2023-04-16 17:10:35.500000
Verifying data_processed/2.csv ...
Found time holes out of tolerance at times:
	2023-04-18 16:08:48.460058 and 2023-04-18 16:17:08.465443
	2023-04-18 16:17:08.465443 and 2023-04-18 16:25:28.471132
	2023-04-18 16:25:28.471132 and 2023-04-18 16:33:48.477

In [7]:
df = pd.concat(dfs)

In [8]:
# make df["time"] hh:mm:ss only
df["time"] = pd.to_datetime(df['time']).dt.floor('S')

In [9]:
df["time"].value_counts()

time
2023-10-12 01:13:35    196
2023-10-12 08:07:35    196
2023-10-12 08:00:35    196
2023-10-12 08:01:35    196
2023-10-12 08:02:35    196
                      ... 
2024-01-01 03:01:32     49
2024-01-01 03:02:32     49
2024-01-01 03:03:32     49
2024-01-01 03:04:32     49
2024-04-02 14:29:30     49
Name: count, Length: 307383, dtype: int64

In [10]:
len(df)

22821456

In [11]:
df.drop_duplicates(inplace=True, keep="first")

In [12]:
len(df)

15061767

In [13]:
df

,time,event_type,channel,value
0,2023-04-16 14:05:35,d,1,0
1,2023-04-16 14:05:35,d,2,1
2,2023-04-16 14:05:35,d,3,0
3,2023-04-16 14:05:35,d,4,0
4,2023-04-16 14:05:35,d,5,0
...,...,...,...,...
42625,2024-04-02 14:29:30,p,5,28
42626,2024-04-02 14:29:30,p,6,31
42627,2024-04-02 14:29:30,p,7,29
42628,2024-04-02 14:29:30,p,8,24


In [14]:
df.sort_values(by=["time", "event_type", "channel"], inplace=True)

In [15]:
verify_df(df)

Found time holes out of tolerance at times:
	2023-04-16 14:07:25 and 2023-04-16 14:32:35
	2023-04-16 15:02:52 and 2023-04-16 15:58:35
	2023-04-16 16:00:25 and 2023-04-16 16:07:35
	2023-04-16 16:37:52 and 2023-04-16 16:54:35
	2023-04-16 16:56:25 and 2023-04-16 17:10:35
	2023-04-16 17:40:52 and 2023-04-18 16:08:48
	2023-04-18 16:08:48 and 2023-04-18 16:17:08
	2023-04-18 16:17:08 and 2023-04-18 16:25:28
	2023-04-18 16:25:28 and 2023-04-18 16:33:48
	2023-04-18 16:33:48 and 2023-04-18 16:42:08
	2023-04-18 16:42:08 and 2023-04-18 16:50:28
	2023-04-18 16:50:28 and 2023-04-18 16:58:48
	2023-04-18 16:58:48 and 2023-04-18 17:07:08
	2023-04-18 17:07:08 and 2023-04-18 17:15:28
	2023-04-18 17:15:28 and 2023-04-18 17:23:48
	2023-04-18 17:23:48 and 2023-04-18 17:32:08
	2023-04-18 17:32:08 and 2023-04-18 17:40:52
	2023-04-18 17:40:52 and 2023-04-18 17:49:12
	2023-04-18 17:49:12 and 2023-04-18 17:57:32
	2023-04-18 17:57:32 and 2023-04-18 18:05:52
	2023-04-18 18:05:52 and 2023-04-18 18:14:12
	2023-04-18

In [16]:
groupped = df.groupby("time")
_groupped = groupped.size()
_groupped[_groupped != 49]

Series([], dtype: int64)

In [17]:
result = df.groupby('time').apply(lambda group: (group['event_type'] == 'p').sum() == 9)
result[result == False]

Series([], dtype: bool)

In [18]:
result = df.groupby('time').apply(lambda group: (group['event_type'] == 'e').sum() == 9)
result[result == False]

Series([], dtype: bool)

In [20]:
result = df.groupby('time').apply(lambda group: (group['event_type'] == 'd').sum() == 31)
result[result == False]

Series([], dtype: bool)

In [22]:
df.query("time >= '2023-09-01'", inplace=True)

In [23]:
df.to_csv("data_processed/fixed.csv", index=False)

# --

In [298]:
load_and_verify_csv(DIR + "preprocessed.csv")

ValueError: Directional events count is not 31 for 2023-09-01 15:22:57.557000

In [13]:
datetime_ranges = []
for i, d in enumerate(dfs):
     datetime_ranges.append((d["time"].min(), d["time"].max(), i))
     print(i, d["time"].min(), d["time"].max(), i, sep="\t")

0	2023-04-16 14:05:35.501119	2023-04-16 17:40:52.381707	0
1	2023-04-16 14:05:35.501000	2023-04-16 17:40:52.383000	1
2	2023-04-18 16:08:48.460058	2023-04-18 23:57:23.751126	2
3	2023-04-18 16:08:48.464000	2023-04-18 23:57:23.755000	3
4	2023-04-19 00:05:43.765000	2023-04-19 19:28:59.531000	4
5	2023-04-19 00:05:43.761000	2023-04-19 23:56:48.497000	5
6	2023-04-19 00:05:43.761317	2023-04-19 23:56:48.496527	6
7	2023-04-20 00:05:08.506000	2023-04-20 01:45:31.504000	7
8	2023-04-20 00:05:08.505619	2023-04-20 01:45:31.503822	8
9	2023-04-24 10:17:08.490000	2023-04-24 23:53:49.329000	9
10	2023-04-24 10:17:08.490000	2023-04-24 19:10:29.009000	10
11	2023-04-24 10:17:08.489712	2023-04-24 23:53:49.328655	11
12	2023-04-25 00:10:29.344000	2023-04-25 14:03:50.311000	12
13	2023-04-25 00:10:29.344318	2023-04-25 14:03:50.310643	13
14	2023-07-07 05:17:03.421000	2023-07-07 05:25:03.554000	14
15	2023-07-07 05:17:03.236559	2023-07-07 05:25:03.368793	15
16	2023-07-09 04:07:03.325445	2023-07-09 04:15:03.455726	16


In [14]:
# check datatime_ranges for overlaps
cnt = 0
for a, b in zip(datetime_ranges, datetime_ranges[1:]):
    if a[1] >= b[0]:
            cnt += 1
            print(cnt, a, b, sep="\t")

# for i in range(len(datetime_ranges)):
#     for j in range(i + 1, len(datetime_ranges)):
#         if datetime_ranges[i][1] >= datetime_ranges[j][0]:
#             print(i, j, datetime_ranges[i], datetime_ranges[j], sep="\t")

1	(Timestamp('2023-04-16 14:05:35.501119'), Timestamp('2023-04-16 17:40:52.381707'), 0)	(Timestamp('2023-04-16 14:05:35.501000'), Timestamp('2023-04-16 17:40:52.383000'), 1)
2	(Timestamp('2023-04-18 16:08:48.460058'), Timestamp('2023-04-18 23:57:23.751126'), 2)	(Timestamp('2023-04-18 16:08:48.464000'), Timestamp('2023-04-18 23:57:23.755000'), 3)
3	(Timestamp('2023-04-19 00:05:43.765000'), Timestamp('2023-04-19 19:28:59.531000'), 4)	(Timestamp('2023-04-19 00:05:43.761000'), Timestamp('2023-04-19 23:56:48.497000'), 5)
4	(Timestamp('2023-04-19 00:05:43.761000'), Timestamp('2023-04-19 23:56:48.497000'), 5)	(Timestamp('2023-04-19 00:05:43.761317'), Timestamp('2023-04-19 23:56:48.496527'), 6)
5	(Timestamp('2023-04-20 00:05:08.506000'), Timestamp('2023-04-20 01:45:31.504000'), 7)	(Timestamp('2023-04-20 00:05:08.505619'), Timestamp('2023-04-20 01:45:31.503822'), 8)
6	(Timestamp('2023-04-24 10:17:08.490000'), Timestamp('2023-04-24 23:53:49.329000'), 9)	(Timestamp('2023-04-24 10:17:08.490000'), 